### 기본 스태킹 모델 p.280~

In [3]:
import numpy as np
import pandas as pd

# 알고리즘 5개 import
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings('ignore')

In [4]:
# 데이터 : 위스콘신 유방암 데이터 로딩
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
# 성능 지표: 정확도
from sklearn.metrics import accuracy_score

In [5]:
# 데이터 로딩 / 분리
cancer_data = load_breast_cancer()

X_data = cancer_data.data
y_label = cancer_data.target

X_train, X_test, y_train, y_test = train_test_split(
    X_data, y_label, test_size = 0.2, random_state = 0
)

In [24]:
# 앙상블에 사용할 알고리즘 객체 생성
knn_clf= KNeighborsClassifier(n_neighbors=4)
rf_clf = RandomForestClassifier(n_estimators=100, random_state=0)
dt_clf= DecisionTreeClassifier() # 기본트리, 파라미터 지정없이 일단 생성(과적합 될것임)
ada_clf = AdaBoostClassifier(n_estimators=100)

In [26]:
# 스태킹에 사용할 객체 생성(최종 메타모델에 사용될 객체)
lr_clf= LogisticRegression(C=10)  # C : 규제

In [27]:
# 개별 학습 진행
knn_clf.fit(X_train, y_train)
rf_clf.fit(X_train,y_train)
dt_clf.fit(X_train,y_train)
ada_clf.fit(X_train,y_train)

AdaBoostClassifier(n_estimators=100)

In [32]:
# 각 모델의 예측값 추출
knn_pred = knn_clf.predict(X_test)
rf_pred = rf_clf.predict(X_test)
dt_pred = dt_clf.predict(X_test)
ada_pred = ada_clf.predict(X_test)
knn_pred.shape, rf_pred.shape, dt_pred.shape, ada_pred.shape
# 스태킹을 하려면, 쌓을 데이터의 차원이 모두 같아야 한다.

((114,), (114,), (114,), (114,))

In [33]:
# 각각의 예측 정확도 출력
print('KNN 정확도 : {0:.4f}'.format(accuracy_score(y_test,knn_pred)))
print('랜덤포레스트 정확도 : {0:.4f}'.format(accuracy_score(y_test,rf_pred)))
print('DecisionTree 정확도 : {0:.4f}'.format(accuracy_score(y_test,dt_pred)))
print('Adaboost 정확도 : {0:.4f}'.format(accuracy_score(y_test,ada_pred)))

KNN 정확도 : 0.9211
랜덤포레스트 정확도 : 0.9649
DecisionTree 정확도 : 0.9123
Adaboost 정확도 : 0.9561


In [34]:
# 각 모델의 예측 결과를 합치기
pred = np.array([knn_pred, rf_pred, dt_pred, ada_pred])

In [36]:
pred.shape

(4, 114)

In [41]:
# 예측결과 행열 위치교환 : np.transpose()
pred = np.transpose(pred)
pred.shape

(114, 4)

In [50]:
# 최종 스태킹 진행 (학습)
lr_clf.fit(pred, y_test)
# 예측/평가
final = lr_clf.predict(pred)
print('최종 정확도: ', accuracy_score(y_test, final))

최종 정확도:  0.9736842105263158


In [43]:
pred

array([[0, 0, 0, 0],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [0, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [0, 0, 0, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [0, 0, 0, 0],
       [0, 0, 1, 1],
       [0, 0, 0, 0],
       [1, 1, 1, 1],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [0, 0, 0, 0],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 0, 0, 0],
       [1, 1, 1, 1],
       [0, 0, 0, 0],
       [1, 1, 1, 1],
       [0, 0, 0, 0],
       [1, 1, 1, 1],
       [0, 0, 0, 0],
       [1, 1, 1, 1],
       [0, 0, 0, 0],
       [1, 1, 1, 1],
       [0, 0, 0, 0],
       [1, 1, 1, 1],
       [0, 0, 0, 0],
       [0, 0, 1, 0],
       [1, 1, 1, 1],
       [0, 0, 0, 0],
       [1, 1, 1, 1],
       [0, 1, 0, 1],
       [0, 0, 0, 0],
       [1, 1, 1, 1],
       [1, 1,

In [44]:
y_test

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0,
       1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1,
       1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0,
       1, 0, 0, 1])

In [45]:
pred.shape, y_test.shape

((114, 4), (114,))

- 위에서 수행한 기본 스태킹에서는, 학습시킨 pred 데이터로 동일하게 예측을 했기 때문에 과적합 가능성이 높다.
- CV 를 이용해 위 문제를 해결해보겠다

### 교차검증(비복원 추출)

In [51]:
from sklearn.model_selection import KFold
# 평균 절대값 오차 (MAE) : 절대값을 이용 오차를 구하는 함수
# 평균 제곱 오차 (MSE): Root mean squre 방법으로 오차를 구하는 함수
from sklearn.metrics import mean_absolute_error

In [103]:
# 개별 기반 모델에서 최종 메타 모델이 사용할 학습 및 테스트 데이터 생성
# 파라미터 : 1.개별 기반 모델, 2. 학습데이터, 3. 학습답, 4.테스트 데이터, 5.폴드 개수  (테스트 답은 나중에 예측에만 쓰이므로 여기서 제외)
def get_stacking_base_datasets(model, X_train_p, y_train_p, X_test_p, folds_p):
    # 지정된 folds_p 값으로 KFold 객체 생성
    kf= KFold(
          n_splits=folds_p
        , shuffle=False
        , random_state=0
    )
    
    # 최종 메타 모델이 사용할 학습/테스트 데이터 저장 배열 생성
#   학습데이터 공간확보 : train_fold_pred = np.zeros((행, 열))
    train_fold_pred = np.zeros((X_train_p.shape[0],1))
#     테스트 데이터 공간 확보
    test_pred = np.zeros((X_test_p.shape[0],folds_p))
    
    print(model.__class__.__name__, 'model 시작')
    # (train_index, valid_index) : fold.split()
    # folder_counter : 폴드의 반복 개수 저장
    for folder_counter, (train_index, valid_index) in enumerate(kf.split(X_train_p)):  #split 함수 의 결과값이 인덱스들로 반환된다.
        print('\t 폴드 세트 : ', folder_counter, '시작')
        # 입력된 학습데이터에서 기반 모델이 학습/예측할 폴드 데이터 세트 추출
        # 학습 데이터 세트 추출
        X_train_v = X_train_p[train_index]
        y_train_v = y_train_p[train_index]   # 데이터든 답이든 어차피 그 위치의 인덱스는 같을테니까
        # 테스트에 사용할 데이터 추출
        X_test_v = X_train_p[valid_index]
        
        # 학습진행
        model.fit(X_train_v, y_train_v)
        
        # 예측값 추출(Train 데이터 안의 validation(내부 테스트 데이터)을 기반으로. )
        train_fold_pred[valid_index, :] = model.predict(X_test_v).reshape(-1,1)
        
        # 테스트 데이터에 대한 예측값 저장
        # 예측값이 폴드 개수만큼 추출됨.( 각 폴드 세트마다 val 이 하나씩 존재하므로)
        test_pred[:, folder_counter] = model.predict(X_test_p)
    # 위 for문에서 나온 총 결과(3개의 결과)의 평균
    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1,1)
    
    return train_fold_pred, test_pred_mean
    

In [104]:
# knn
knn_train, knn_test = get_stacking_base_datasets(
    knn_clf, X_train, y_train, X_test, 7
)

# knn_test 는 결과의 평균값이다.

KNeighborsClassifier model 시작
	 폴드 세트 :  0 시작
	 폴드 세트 :  1 시작
	 폴드 세트 :  2 시작
	 폴드 세트 :  3 시작
	 폴드 세트 :  4 시작
	 폴드 세트 :  5 시작
	 폴드 세트 :  6 시작


In [106]:
knn_train

numpy.ndarray

In [107]:
# Random Forest
rf_train, rf_test = get_stacking_base_datasets(
    rf_clf, X_train, y_train, X_test, 7
)

RandomForestClassifier model 시작
	 폴드 세트 :  0 시작
	 폴드 세트 :  1 시작
	 폴드 세트 :  2 시작
	 폴드 세트 :  3 시작
	 폴드 세트 :  4 시작
	 폴드 세트 :  5 시작
	 폴드 세트 :  6 시작


In [108]:
# Decision Tree
dt_train, dt_test = get_stacking_base_datasets(
    dt_clf, X_train, y_train, X_test, 7
)

DecisionTreeClassifier model 시작
	 폴드 세트 :  0 시작
	 폴드 세트 :  1 시작
	 폴드 세트 :  2 시작
	 폴드 세트 :  3 시작
	 폴드 세트 :  4 시작
	 폴드 세트 :  5 시작
	 폴드 세트 :  6 시작


In [109]:
# Adaboost
ada_train, ada_test = get_stacking_base_datasets(
    ada_clf, X_train, y_train, X_test, 7
)

AdaBoostClassifier model 시작
	 폴드 세트 :  0 시작
	 폴드 세트 :  1 시작
	 폴드 세트 :  2 시작
	 폴드 세트 :  3 시작
	 폴드 세트 :  4 시작
	 폴드 세트 :  5 시작
	 폴드 세트 :  6 시작


In [110]:
# knn, rf, dt, ada 데이터 하나로 Stacking: concat으로 이어붙이기
stack_X_train = np.concatenate((knn_train, rf_train, dt_train, ada_train) ,axis=1)
stack_X_test = np.concatenate((knn_test, rf_test, dt_test, ada_test), axis=1)

In [114]:
# 원래 학습 데이터 : X_train, 원래 테스트 데이터 : X_test
X_train.shape, X_test.shape

((455, 30), (114, 30))

In [115]:
# 스태킹 처리된 학습데이터 /테스트데이터
stack_X_train.shape, stack_X_test.shape
# 4개의 모델을 썼으므로, 각 모델당 예측 결과값이 들어가게 됨.

((455, 4), (114, 4))

In [116]:
# 최종 모델 학습
lr_clf.fit(stack_X_train, y_train)
# 예측값 추출
stack_pred = lr_clf.predict(stack_X_test)

stack_accu = accuracy_score(y_test, stack_pred)
print('최종 정확도 : {0:.6f}'.format(stack_accu))

최종 정확도 : 0.973684


정확도가 이전과 동일